In [ ]:
import pandas as pd
import csv
from tqdm import tqdm
import ast

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
root_path = ''
path_out = root_path + ''

In [ ]:
from evaluation_dataset import EvaluationDataset

In [ ]:
from evaluate import evaluate

In [ ]:
K = [1, 3, 5, 10, 15, 20, 30]

In [ ]:
collab_path = root_path + 'results/evaluation_data_deepconn_fm.csv'
collab_df = pd.read_csv(collab_path)

In [ ]:
bow_path = root_path + 'results/evaluation_data_node2vec_update.csv'
bow_df = pd.read_csv(bow_path)

In [ ]:
def parse_input_list(list_string):
    return ast.literal_eval(list_string)

In [ ]:
def r(ranking, k):
    return 1.0 / (k + ranking)
    
def rrf_score(ranking_collab, ranking_bow, k=60):
    r1 = r(ranking_collab, k)
    r2 = r(ranking_bow, k)
    return sum([r1, r2])

In [ ]:
evaluation_dataset = EvaluationDataset()

In [ ]:
with open(path_out, mode='w') as f:
    writer = csv.writer(f)
    writer.writerow(['author_id', 'k', 'hits_at_k', 'interacted_count', 'precision', 'recall', 'AP', 'documents', 'ranking'])

In [ ]:
pos = 0
evaluation_dataset.current_line = 0
for author_id, user_rep, article_ids, le, y_t in tqdm(evaluation_dataset, total=len(evaluation_dataset)):
    ranking_collab = parse_input_list(collab_df.iloc[pos]['ranking'])
    ranking_bow = parse_input_list(bow_df.iloc[pos]['ranking'])
    
    ranking_collab = [int(x) for x in ranking_collab]
    ranking_bow = [int(x) for x in ranking_bow]
    scores = []
    y = []
    for article_id in article_ids:
        if article_id == article_ids[0]:
            y.append(1)
        else:
            y.append(0)
        interacted_pos_collab = ranking_collab.index(article_id)
        interacted_pos_bow = ranking_bow.index(article_id)
        new_score = rrf_score(interacted_pos_collab, interacted_pos_bow)
        scores.append(new_score)
        
    out = pd.DataFrame({'v': scores, 'flag': y, 'article_ids': article_ids})
    out = out.sort_values(by='v', ascending=False)
    article_ids_ranked = out['article_ids'].tolist()
    y_true = out['flag'].tolist()
    
    interacted_items_count_testset = 1
    
    for k in K:
        hits_at_k = sum(y_true[:k])
        interacted_items_count_testset, precision, recall = evaluate(interacted_items_count_testset, hits_at_k, k)
        precisions_at = []
        for i, el in enumerate(y_true[:k]):
            precisions_at.append(sum(y_true[:i + 1]) / (i + 1))

        AP_at_k = sum(precisions_at) / k

        with open(path_out, 'a', encoding='utf-8') as f:
            writer = csv.writer(f)
            # ['author_id', 'k', 'hits_at_k', 'interacted_count', 'precision', 'recall']
            writer.writerow(
                [author_id, k, hits_at_k, interacted_items_count_testset, precision, recall, round(AP_at_k, 4),
                 len(out), article_ids_ranked])

    pos += len(K)
    
    

In [ ]:
sum(1,2)

In [ ]:
test_df = pd.read_csv(root_path + 'results/evaluation_fusion.csv')

In [ ]:
pos += len(K) 

In [ ]:
evaluation_dataset.test_df.head()

In [ ]:
evaluation_dataset.test_df[evaluation_dataset.test_df['article_id'] == 4419688.0]